In [ ]:
import numpy as np
import sys
import tensorflow as tf
from keras.losses import Loss
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
import pandas as pd
import datetime
import time
import io
from sklearn.model_selection import train_test_split

In [ ]:
import torch
import typing

from pathlib import Path

from torchvision import datasets

from sklearn import model_selection


def lambda_top_func(mu, k, y, alpha):
    m = y.shape[0]
    r = (y[k:] - mu).sum(dim=0)
    return mu + r.div(m * (alpha + 1) - k)


def lambda_bottom_func(mu, k, y, alpha):
    m = y.shape[0]
    r = (y[:k] - mu).sum(dim=0)
    return mu + r.div(m * alpha + k)


def alpha_fn(pi, lambda_):
    return (pi * lambda_) ** -1 + 1.0 - lambda_ ** -1


def beta_fn(pi, lambda_):
    return lambda_ * (pi) ** -1 + 1.0 - lambda_


def policy_risk(pi, y1, y0):
    return (pi * y1 + (1 - pi) * y0).mean()

def complete_propensity(x, u, gamma, beta=0.75):
    logit = beta * x + 0.5
    nominal = (1 + np.exp(-logit)) ** -1
    alpha = alpha_fn(nominal, gamma)
    beta = beta_fn(nominal, gamma)
    return (u / alpha) + ((1 - u) / beta)


def f_mu(x, t, u, theta=4.0):
    mu = (
        (2 * t - 1) * x
        + (2.0 * t - 1)
        - 2 * np.sin((4 * t - 2) * x)
        - (theta * u - 2) * (1 + 0.5 * x)
    )
    return mu


def linear_normalization(x, new_min, new_max):
    return (x - x.min()) * (new_max - new_min) / (x.max() - x.min()) + new_min


class HCMNIST(datasets.MNIST):
    def __init__(
        self,
        root: str,
        gamma_star: float,
        split: str = "train",
        mode: str = "mu",
        p_u: str = "bernoulli",
        theta: float = 4.0,
        beta: float = 0.75,
        sigma_y: float = 1.0,
        domain: float = 2.0,
        seed: int = 1331,
        transform: typing.Optional[typing.Callable] = None,
        target_transform: typing.Optional[typing.Callable] = None,
        download: bool = True,
    ) -> None:
        train = split == "train" or split == "valid"
        root = Path.home() / "quince_datasets" if root is None else Path(root)
        self.__class__.__name__ = "MNIST"
        super(HCMNIST, self).__init__(
            root,
            train=train,
            transform=transform,
            target_transform=target_transform,
            download=download,
        )
        self.data = self.data.view(len(self.targets), -1).numpy()
        self.targets = self.targets.numpy()

        if train:
            (
                data_train,
                data_valid,
                targets_train,
                targets_valid,
            ) = model_selection.train_test_split(
                self.data, self.targets, test_size=0.3, random_state=seed
            )
            self.data = data_train if split == "train" else data_valid
            self.targets = targets_train if split == "train" else targets_valid

        self.mode = mode
        self.dim_input = [1, 28, 28]
        self.dim_treatment = 1
        self.dim_output = 1

        self.phi_model = fit_phi_model(
            root=root, edges=torch.arange(-domain, domain + 0.1, (2 * domain) / 10),
        )

        size = (self.__len__(), 1)
        rng = np.random.RandomState(seed=seed)
        if p_u == "bernoulli":
            self.u = rng.binomial(1, 0.5, size=size).astype("float32")
        elif p_u == "uniform":
            self.u = rng.uniform(size=size).astype("float32")
        elif p_u == "beta_bi":
            self.u = rng.beta(0.5, 0.5, size=size).astype("float32")
        elif p_u == "beta_uni":
            self.u = rng.beta(2, 5, size=size).astype("float32")
        else:
            raise NotImplementedError(f"{p_u} is not a supported distribution")

        phi = self.phi
        self.pi = (
            complete_propensity(x=phi, u=self.u, gamma=gamma_star, beta=beta)
            .astype("float32")
            .ravel()
        )
        self.t = rng.binomial(1, self.pi).astype("float32")
        eps = (sigma_y * rng.normal(size=self.t.shape)).astype("float32")
        self.mu0 = (
            f_mu(x=phi, t=0.0, u=self.u, theta=theta).astype("float32").ravel()
        )
        self.mu1 = (
            f_mu(x=phi, t=1.0, u=self.u, theta=theta).astype("float32").ravel()
        )
        self.y0 = self.mu0 + eps
        self.y1 = self.mu1 + eps
        self.y = self.t * self.y1 + (1 - self.t) * self.y0
        self.tau = self.mu1 - self.mu0
        self.y_mean = np.array([0.0], dtype="float32")
        self.y_std = np.array([1.0], dtype="float32")

    def __getitem__(self, index):
        x = ((self.data[index].astype("float32") / 255.0) - 0.1307) / 0.3081
        t = self.t[index : index + 1]
        if self.mode == "pi":
            return x, t
        elif self.mode == "mu":
            return np.hstack([x, t]), self.y[index : index + 1]
        else:
            raise NotImplementedError(
                f"{self.mode} not supported. Choose from 'pi'  for propensity models or 'mu' for expected outcome models"
            )

    @property
    def phi(self):
        x = ((self.data.astype("float32") / 255.0) - 0.1307) / 0.3081
        z = np.zeros_like(self.targets.astype("float32"))
        for k, v in self.phi_model.items():
            ind = self.targets == k
            x_ind = x[ind].reshape(ind.sum(), -1)
            means = x_ind.mean(axis=-1)
            z[ind] = linear_normalization(
                np.clip((means - v["mu"]) / v["sigma"], -1.4, 1.4), v["lo"], v["hi"]
            )
        return np.expand_dims(z, -1)

    @property
    def x(self):
        return ((self.data.astype("float32") / 255.0) - 0.1307) / 0.3081


def fit_phi_model(root, edges):
    ds = datasets.MNIST(root=root)
    data = (ds.data.float().div(255) - 0.1307).div(0.3081).view(len(ds), -1)
    model = {}
    digits = torch.unique(ds.targets)
    for i, digit in enumerate(digits):
        lo, hi = edges[i : i + 2]
        ind = ds.targets == digit
        data_ind = data[ind].view(ind.sum(), -1)
        means = data_ind.mean(dim=-1)
        mu = means.mean()
        sigma = means.std()
        model.update(
            {
                digit.item(): {
                    "mu": mu.item(),
                    "sigma": sigma.item(),
                    "lo": lo.item(),
                    "hi": hi.item(),
                }
            }
        )
    return model

In [ ]:
def get_HMINIST_data(sim_num):
  obj = HCMNIST(root = "./",gamma_star = 1.1, seed=sim_num)
  # Assuming X is a matrix of size (42000, 784)
  # Assuming indices is the array of generated indices
  indices = np.random.choice(42000, size=3000, replace=False)
  # Select subrows from X using the generated indices

  X = obj.x[indices]
  Y_factual = obj.y[indices]
  T = obj.t[indices]
  mu_0 = obj.mu0[indices]
  mu_1 = obj.mu1[indices]

  X_train, X_test, Y_factual_train, _, T_train, _, mu_0_train, mu_0_test, mu_1_train, mu_1_test = train_test_split(X, Y_factual, T, mu_0, mu_1, test_size=0.1, random_state=sim_num)

  X_train = X_train.astype('float32')
  Y_factual_train = Y_factual_train.astype('float32') #most GPUs only compute 32-bit floats
  T_train = T_train.astype('float32')
  mu_0_train = mu_0_train.astype('float32')
  mu_1_train = mu_1_train.astype('float32')

  X_test = X_test.astype('float32')
  mu_0_test = mu_0_test.astype('float32')
  mu_1_test = mu_1_test.astype('float32')

  data_train={'x':X_train,'y':Y_factual_train,'t':T_train,'mu_0':mu_0_train,'mu_1':mu_1_train}
  data_train['t']=data_train['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
  data_train['y']=data_train['y'].reshape(-1,1)
  data_train['mu_0']=data_train['mu_0'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
  data_train['mu_1']=data_train['mu_1'].reshape(-1,1)

  #rescaling y between 0 and 1 often makes training of DL regressors easier
  data_train['y_scaler'] = StandardScaler().fit(data_train['y'])
  data_train['ys'] = data_train['y_scaler'].transform(data_train['y'])

  data_test={'x':X_test,'mu_0':mu_0_test,'mu_1':mu_1_test}
  data_test['y_scaler'] = data_train['y_scaler']
  data_test['mu_0']=data_test['mu_0'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
  data_test['mu_1']=data_test['mu_1'].reshape(-1,1)

  return data_train, data_test

In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.losses import Loss
class EpsilonLayer(Layer):

    def __init__(self):
        super(EpsilonLayer, self).__init__()

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.epsilon = self.add_weight(name='epsilon',
                                       shape=[1, 1],
                                       initializer='RandomNormal',
                                       #  initializer='ones',
                                       trainable=True)
        super(EpsilonLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, **kwargs):
        #note there is only one epsilon were just duplicating it for conformability
        return self.epsilon * tf.ones_like(inputs)[:, 0:1]

def make_dragonnet(input_dim, reg_l2):

    x = Input(shape=(input_dim,), name='input')
    # representation
    phi = Dense(units=512, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=256, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=128, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=64, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=32, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=64, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=32, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #propensity prediction
    #Note that the activation is actually sigmoid, but we will squish it in the loss function for numerical stability reasons
    t_predictions = Dense(units=1,activation=None,name='t_prediction')(phi)
    #Although the epsilon layer takes an input, it really just houses a free parameter.
    epsilons = EpsilonLayer()(t_predictions)
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,t_predictions,epsilons,phi])
    model = Model(inputs=x, outputs=concat_pred)
    return model


In [ ]:
class Base_Loss(Loss):
    #initialize instance attributes
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha
        self.name='standard_loss'

    def split_pred(self,concat_pred):
        #generic helper to make sure we dont make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['phi'] = concat_pred[:, 3:]
        return preds

    #for logging purposes only
    def treatment_acc(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        #Since this isn't used as a loss, I've used tf.reduce_mean for interpretability
        return tf.reduce_mean(binary_accuracy(t_true, tf.math.sigmoid(p['t_pred']), threshold=0.5))

    def treatment_bce(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        lossP = tf.reduce_sum(binary_crossentropy(t_true,p['t_pred'],from_logits=True))
        return lossP

    def regression_loss(self,concat_true,concat_pred):
        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - p['y0_pred']))
        loss1 = tf.reduce_sum(t_true * tf.square(y_true - p['y1_pred']))
        return loss0+loss1

    def standard_loss(self,concat_true,concat_pred):
        lossR = self.regression_loss(concat_true,concat_pred)
        lossP = self.treatment_bce(concat_true,concat_pred)
        return lossR + self.alpha * lossP

    #compute loss
    def call(self, concat_true, concat_pred):
        return self.standard_loss(concat_true,concat_pred)

In [ ]:
class TarReg_Loss(Base_Loss):
    #initialize instance attributes
    def __init__(self, alpha=1,beta=1):
        super().__init__()
        self.alpha = alpha
        self.beta=beta
        self.name='tarreg_loss'

    def split_pred(self,concat_pred):
        #generic helper to make sure we dont make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3] #we're moving epsilon into slot three
        preds['phi'] = concat_pred[:, 4:]
        return preds

    def calc_hstar(self,concat_true,concat_pred):
        #step 2 above
        p=self.split_pred(concat_pred)
        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]

        t_pred = tf.math.sigmoid(concat_pred[:, 2])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        y_pred = t_true * p['y1_pred'] + (1 - t_true) * p['y0_pred']

        #calling it cc for "clever covariate" as in SuperLearner TMLE literature
        cc = t_true / t_pred - (1 - t_true) / (1 - t_pred)
        h_star = y_pred + p['epsilon'] * cc
        return h_star

    def call(self,concat_true,concat_pred):
        y_true = concat_true[:, 0]

        standard_loss=self.standard_loss(concat_true,concat_pred)
        h_star=self.calc_hstar(concat_true,concat_pred)
        #step 3 above
        targeted_regularization = tf.reduce_sum(tf.square(y_true - h_star))

        # final
        loss = standard_loss + self.beta * targeted_regularization
        return loss

In [ ]:

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class Eval_metrics_train():
    def __init__(self,data):
        self.data=data #feed the callback the full dataset
        #needed for PEHEnn; Called in self.find_ynn
        self.data['o_idx']=tf.range(self.data['t'].shape[0])
        self.data['c_idx']=self.data['o_idx'][self.data['t'].squeeze()==0] #These are the indices of the control units
        self.data['t_idx']=self.data['o_idx'][self.data['t'].squeeze()==1] #These are the indices of the treated units

    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
        preds['y1_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3]
        preds['phi'] = concat_pred[:, 4:]
        return preds

    def ATE_absolute_error(self,concat_pred):
        p = self.split_pred(concat_pred)
        ATT_pred = tf.gather(params=self.data['y'], indices=self.data['t_idx']) - tf.gather(params=p['y0_pred'], indices=self.data['t_idx'])
        ATU_pred = tf.gather(params=p['y1_pred'], indices=self.data['c_idx']) - tf.gather(params=self.data['y'], indices=self.data['c_idx'])
        ATE_pred = tf.reduce_mean(tf.concat([ATT_pred,ATU_pred], axis=0)) #stitch em back up!
        ATE_actual = tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        return tf.abs(ATE_actual- ATE_pred)

    def ITE_RMSE_error(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        y_1_treated_group = tf.gather(params=self.data['y'], indices=self.data['t_idx'])
        y_0_treated_group = tf.gather(params=p['y0_pred'], indices=self.data['t_idx'])

        mu_1_treated_group = tf.gather(params=self.data['mu_1'], indices=self.data['t_idx'])
        mu_0_treated_group = tf.gather(params=self.data['mu_0'], indices=self.data['t_idx'])


        treat_grp_error = (y_1_treated_group - y_0_treated_group) - (mu_1_treated_group - mu_0_treated_group)

        y_1_control_group = tf.gather(params=p['y1_pred'], indices=self.data['c_idx'])
        y_0_control_group = tf.gather(params=self.data['y'], indices=self.data['c_idx'])

        mu_1_control_group = tf.gather(params=self.data['mu_1'], indices=self.data['c_idx'])
        mu_0_control_group = tf.gather(params=self.data['mu_0'], indices=self.data['c_idx'])

        control_grp_error = (y_1_control_group - y_0_control_group) - (mu_1_control_group - mu_0_control_group)


        ITE_error = tf.concat([treat_grp_error, control_grp_error], axis=0)
        ITE_RMSE_error = tf.sqrt(tf.reduce_mean(tf.square(ITE_error)))
        return ITE_RMSE_error

    def compute_hstar(self,y0_pred,y1_pred,t_pred,t_true,epsilons):
        #helper for calculating the targeted regularization cate
        y_pred = t_true * y1_pred + (1 - t_true) * y0_pred
        cc = t_true / t_pred - (1 - t_true) / (1 - t_pred)
        h_star = y_pred + epsilons * cc
        return h_star

    def TARREG_ATE_absolute_error(self,concat_pred):
        #Final calculation of Targeted Regularization loss
        p = self.split_pred(concat_pred)
        t_pred = tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        hstar_0=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.zeros_like(p['epsilon']),p['epsilon'])
        hstar_1=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.ones_like(p['epsilon']),p['epsilon'])
        ate_true=tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        ate_tarreg_pred = tf.reduce_mean(hstar_1-hstar_0)
        return tf.abs(ate_true- ate_tarreg_pred)

In [ ]:

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class Eval_metrics_test():
    def __init__(self,data):
        self.data=data #feed the callback the full dataset

    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
        preds['y1_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3]
        preds['phi'] = concat_pred[:, 4:]
        return preds


    def PEHE(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        cate_err=tf.reduce_mean( tf.square( ( (self.data['mu_1']-self.data['mu_0']) - (p['y1_pred']-p['y0_pred']) ) ) )
        return tf.sqrt(cate_err)

    def PEHE_tareg(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        t_pred = tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        hstar_0=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.zeros_like(p['epsilon']),p['epsilon'])
        hstar_1=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.ones_like(p['epsilon']),p['epsilon'])

        cate_err=tf.reduce_mean( tf.square( ( (self.data['mu_1']-self.data['mu_0']) - (hstar_1-hstar_0) ) ) )
        return tf.sqrt(cate_err)


    def compute_hstar(self,y0_pred,y1_pred,t_pred,t_true,epsilons):
        #helper for calculating the targeted regularization cate
        y_pred = t_true * y1_pred + (1 - t_true) * y0_pred
        cc = t_true / t_pred - (1 - t_true) / (1 - t_pred)
        h_star = y_pred + epsilons * cc
        return h_star

    def TARREG_ATE_absolute_error(self,concat_pred):
        #Final calculation of Targeted Regularization loss
        p = self.split_pred(concat_pred)
        t_pred = tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        hstar_0=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.zeros_like(p['epsilon']),p['epsilon'])
        hstar_1=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.ones_like(p['epsilon']),p['epsilon'])
        ate_true=tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        print(tf.shape(hstar_1-hstar_0))
        ate_tarreg_pred = tf.reduce_mean(hstar_1-hstar_0)
        return tf.abs(ate_true- ate_tarreg_pred)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam
import io
from IPython.display import display, clear_output
#Colab command to allow us to run Colab in TF2
%load_ext tensorboard
# data=get_news_data(1)
val_split=0.20
batch_size=4000
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
!rm -rf ./logs_dragonnet/
sim_evals = []
for j in range(0,50):
  clear_output(wait=True)
  print(j+1)
  data_train, data_test = get_HMINIST_data(j+1)
  yt = np.concatenate([data_train['ys'], data_train['t']], 1)

  # Clear any logs from previous runs

  start_time = time.time()

  log_dir = "logs_dragonnet/fit/" +str(j)+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
  file_writer.set_as_default()
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)


  sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=20, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0),
        tensorboard_callback]

  sgd_lr = 0.0000001
  momentum = 0.9

  dragonnet_model=make_dragonnet(data_train['x'].shape[1],.01)
  tarreg_loss=TarReg_Loss(alpha=1)

  dragonnet_model.compile(optimizer=SGD(learning_rate=sgd_lr, momentum=momentum, nesterov=True),
                      loss=tarreg_loss,
                 metrics=[tarreg_loss,tarreg_loss.regression_loss,tarreg_loss.treatment_acc])

  dragonnet_model.fit(x=data_train['x'],y=yt,
                 callbacks=sgd_callbacks,
                  validation_split=val_split,
                  epochs=100000,
                  batch_size=batch_size,
                  verbose=verbose)
  end_time = time.time()

  elapsed_time_seconds = end_time - start_time
  # Convert elapsed time to minutes
  elapsed_time_minutes = elapsed_time_seconds / 60

  Evaluation_metrics_insample = Eval_metrics_train(data_train)
  concat_pred_insample = dragonnet_model.predict(data_train['x'])
  ATE_abs_insample = Evaluation_metrics_insample.ATE_absolute_error(concat_pred_insample)
  ATE_TARREG_insample = Evaluation_metrics_insample.TARREG_ATE_absolute_error(concat_pred_insample)
  ITE_RMSE_insample = Evaluation_metrics_insample.ITE_RMSE_error(concat_pred_insample)

  Evaluation_metrics_outsample = Eval_metrics_test(data_test)
  concat_pred_outsample = dragonnet_model.predict(data_test['x'])
  ATE_TARREG_outsample = Evaluation_metrics_outsample.TARREG_ATE_absolute_error(concat_pred_outsample)
  PEHE_outsample = Evaluation_metrics_outsample.PEHE(concat_pred_outsample)
  PEHE_tareg_outsample = Evaluation_metrics_outsample.PEHE_tareg(concat_pred_outsample)

  metrics = [ATE_abs_insample.numpy(), ATE_TARREG_insample.numpy(), ITE_RMSE_insample.numpy(), ATE_TARREG_outsample.numpy(), PEHE_outsample.numpy(), PEHE_tareg_outsample.numpy(), elapsed_time_minutes]
  sim_evals.append(metrics)

50
Epoch 1/100000
1/1 [==============================] - 3s 3s/step - loss: 4933.2227 - tarreg_loss: 4928.4834 - regression_loss: 2449.0789 - treatment_acc: 0.4347 - val_loss: 1332.0057 - val_tarreg_loss: 1327.2664 - val_regression_loss: 655.5977 - val_treatment_acc: 0.4593 - lr: 1.0000e-07
Epoch 2/100000
1/1 [==============================] - 0s 118ms/step - loss: 4890.4194 - tarreg_loss: 4885.6802 - regression_loss: 2430.0205 - treatment_acc: 0.4333 - val_loss: 1315.3153 - val_tarreg_loss: 1310.5759 - val_regression_loss: 648.1030 - val_treatment_acc: 0.4574 - lr: 1.0000e-07
Epoch 3/100000
1/1 [==============================] - 0s 123ms/step - loss: 4836.6123 - tarreg_loss: 4831.8730 - regression_loss: 2406.0166 - treatment_acc: 0.4333 - val_loss: 1297.1826 - val_tarreg_loss: 1292.4434 - val_regression_loss: 639.9438 - val_treatment_acc: 0.4574 - lr: 1.0000e-07
Epoch 4/100000
1/1 [==============================] - 0s 120ms/step - loss: 4777.5503 - tarreg_loss: 4772.8110 - regression_

In [ ]:
sim_evals = np.asarray(sim_evals)
# with open('eval_metrics_news_20.npy', 'wb') as f:
#     np.save(f,sim_evals)

In [ ]:
# [ATE_abs_insample.numpy(), ATE_TARREG_insample.numpy(), ITE_RMSE_insample.numpy(), ATE_TARREG_outsample.numpy(), PEHE_outsample.numpy(), PEHE_tareg_outsample.numpy(), elapsed_time_minutes

print(" Insample ATE error for Dragonnet(mean over 50) =", round(np.mean(sim_evals[:,0]),2),"+-",round((np.std(sim_evals[:,0], ddof=1) / np.sqrt(np.size(sim_evals[:,0]))),2))
print(" Insample ATE targ_reg error for Dragonnet(mean over 50) =", round(np.mean(sim_evals[:,1]),2),"+-",round((np.std(sim_evals[:,1], ddof=1) / np.sqrt(np.size(sim_evals[:,1]))),2))
print(" Insample ITE_RMSE error for Dragonnet(mean over 50) =", round(np.mean(sim_evals[:,2]),2),"+-",round((np.std(sim_evals[:,2], ddof=1) / np.sqrt(np.size(sim_evals[:,2]))),2))
print(" Outsample ATE targ_reg error for Dragonnet(mean over 50) =", round(np.mean(sim_evals[:,3]),2),"+-",round((np.std(sim_evals[:,3], ddof=1) / np.sqrt(np.size(sim_evals[:,3]))),2))
print(" Outsample PEHE error for Dragonnet(mean over 50) =", round(np.mean(sim_evals[:,4]),2),"+-",round((np.std(sim_evals[:,4], ddof=1) / np.sqrt(np.size(sim_evals[:,4]))),2))
print(" Outsample PEHE targ_reg error for Dragonnet(mean over 50) =", round(np.mean(sim_evals[:,5]),2),"+-",round((np.std(sim_evals[:,5], ddof=1) / np.sqrt(np.size(sim_evals[:,5]))),2))
print(" Wall time for Dragonnet(sum over 50) =", np.sum(sim_evals[:,6]))

NameError: ignored

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()